In [3]:
!pip install ta
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dropout,GRU
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

df = pd.read_csv("./[경기관련소비재]현대차.csv")
df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.columns = df_columns
df = df.sort_values('Date')
df['Date']= df['Date'].astype('str')
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
    df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

df.dropna(inplace=True)


import ta

H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

scaler1 = MinMaxScaler()

temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

scaled_df.insert(3, 'Close',temp)
scaled_df

df = scaled_df



import numpy as np

def make_dataset(feature, label, window_size = 20):
  
  feature_list = []
  label_list = []
  
  for i in range(len(feature) - window_size) :
    feature_list.append(feature.iloc[i:i+window_size])
    label_list.append(label.iloc[i+window_size])
  
  return np.array(feature_list), np.array(label_list)

WINDOW_SIZE=20

result = []
predict = []


####최근 20개의 데이터만
pred_feature = df[-WINDOW_SIZE :]
pred_feature = pred_feature.to_numpy()

pred_feature = pred_feature.reshape((1,20,-1))
# pred_feature.shape

import random as rn
import tensorflow as tf


for i in range(1, len(df)//1000+2):

        # 데이터 불러오기
        df_copy = df
     
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()
        model.add(GRU(64, 
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences = True,
                      activation='relu'))
        model.add(GRU(64, return_sequences=False, stateful=False))
        model.add(Dense(1))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        
        

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)



C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
34/34 [==============================] - ETA: 0s - loss: 0.0048
Epoch 1: val_loss improved from inf to 0.06430, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 37s 287ms/step - loss: 0.0048 - val_loss: 0.0643
Epoch 2/100
32/34 [===========================>..] - ETA: 0s - loss: 7.1622e-04
Epoch 2: val_loss improved from 0.06430 to 0.02656, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 2s 47ms/step - loss: 7.0067e-04 - val_loss: 0.0266
Epoch 3/100
33/34 [============================>.] - ETA: 0s - loss: 3.8327e-04
Epoch 3: val_loss improved from 0.02656 to 0.00997, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 2s 53ms/step - loss: 3.8242e-04 - val_loss: 0.0100
Epoch 4/100
33/34 [============================>.] - ETA: 0s - loss: 4.2772e-04
Epoch 4: val_loss did not improve from 0.00997
34/34 [==============================] - 1s 37ms/step - loss: 4.3189e-04 - val_loss: 0.0139

69/69 [==============================] - ETA: 0s - loss: 7.2393e-04
Epoch 2: val_loss improved from 0.00287 to 0.00026, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 4s 59ms/step - loss: 7.2393e-04 - val_loss: 2.5800e-04
Epoch 3/100
67/69 [============================>.] - ETA: 0s - loss: 2.9250e-04
Epoch 3: val_loss improved from 0.00026 to 0.00019, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 10s 142ms/step - loss: 2.9383e-04 - val_loss: 1.8634e-04
Epoch 4/100
68/69 [============================>.] - ETA: 0s - loss: 2.7915e-04
Epoch 4: val_loss improved from 0.00019 to 0.00016, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 5s 75ms/step - loss: 2.7826e-04 - val_loss: 1.6225e-04
Epoch 5/100
67/69 [============================>.] - ETA: 0s - loss: 2.2997e-04
Epoch 5: val_loss did not improve from 0.00016
69/69 [==============================] - 2s 31ms/step - loss: 2.2953e-04 - val_

104/104 [==============================] - 6s 56ms/step - loss: 3.3916e-04 - val_loss: 1.5804e-04
Epoch 8/100
104/104 [==============================] - ETA: 0s - loss: 3.3176e-04
Epoch 8: val_loss did not improve from 0.00016
104/104 [==============================] - 5s 48ms/step - loss: 3.3176e-04 - val_loss: 1.6835e-04
Epoch 9/100
103/104 [============================>.] - ETA: 0s - loss: 3.2449e-04
Epoch 9: val_loss did not improve from 0.00016
104/104 [==============================] - 5s 49ms/step - loss: 3.2346e-04 - val_loss: 1.5982e-04
Epoch 10/100
104/104 [==============================] - ETA: 0s - loss: 4.2408e-04
Epoch 10: val_loss improved from 0.00016 to 0.00014, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 5s 44ms/step - loss: 4.2408e-04 - val_loss: 1.3887e-04
Epoch 11/100
102/104 [============================>.] - ETA: 0s - loss: 3.6102e-04
Epoch 11: val_loss did not improve from 0.00014
104/104 [==============================] - 

173/174 [============================>.] - ETA: 0s - loss: 2.3676e-04
Epoch 3: val_loss did not improve from 0.00037
174/174 [==============================] - 7s 41ms/step - loss: 2.3608e-04 - val_loss: 7.3052e-04
Epoch 4/100
174/174 [==============================] - ETA: 0s - loss: 2.5518e-04
Epoch 4: val_loss did not improve from 0.00037
174/174 [==============================] - 6s 34ms/step - loss: 2.5518e-04 - val_loss: 5.4911e-04
Epoch 5/100
174/174 [==============================] - ETA: 0s - loss: 2.0940e-04
Epoch 5: val_loss did not improve from 0.00037
174/174 [==============================] - 6s 33ms/step - loss: 2.0940e-04 - val_loss: 4.1483e-04
Epoch 6/100
174/174 [==============================] - ETA: 0s - loss: 2.1650e-04
Epoch 6: val_loss did not improve from 0.00037
174/174 [==============================] - 5s 30ms/step - loss: 2.1650e-04 - val_loss: 7.6331e-04
Epoch 7/100
172/174 [============================>.] - ETA: 0s - loss: 2.1030e-04
Epoch 7: val_loss impr

...

277/279 [============================>.] - ETA: 0s - loss: 1.9965e-05
Epoch 17: val_loss improved from 0.00043 to 0.00034, saving model to model\tmp_checkpoint.h5
279/279 [==============================] - 7s 24ms/step - loss: 1.9924e-05 - val_loss: 3.4495e-04
Epoch 18/100
279/279 [==============================] - ETA: 0s - loss: 1.8495e-05
Epoch 18: val_loss improved from 0.00034 to 0.00031, saving model to model\tmp_checkpoint.h5
279/279 [==============================] - 7s 25ms/step - loss: 1.8495e-05 - val_loss: 3.1255e-04
Epoch 19/100
278/279 [============================>.] - ETA: 0s - loss: 2.1154e-05
Epoch 19: val_loss did not improve from 0.00031
279/279 [==============================] - 7s 26ms/step - loss: 2.1161e-05 - val_loss: 3.8714e-04
Epoch 20/100
277/279 [============================>.] - ETA: 0s - loss: 2.2344e-05
Epoch 20: val_loss did not improve from 0.00031
279/279 [==============================] - 7s 26ms/step - loss: 2.2368e-05 - val_loss: 4.0513e-04
Epoch 2

312/314 [============================>.] - ETA: 0s - loss: 1.5504e-05
Epoch 23: val_loss did not improve from 0.00038
314/314 [==============================] - 7s 23ms/step - loss: 1.5533e-05 - val_loss: 7.1714e-04
Epoch 24/100
312/314 [============================>.] - ETA: 0s - loss: 1.4275e-05
Epoch 24: val_loss improved from 0.00038 to 0.00031, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 7s 21ms/step - loss: 1.4280e-05 - val_loss: 3.1022e-04
Epoch 25/100
314/314 [==============================] - ETA: 0s - loss: 1.5978e-05
Epoch 25: val_loss improved from 0.00031 to 0.00031, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 7s 23ms/step - loss: 1.5978e-05 - val_loss: 3.0888e-04
Epoch 26/100
313/314 [============================>.] - ETA: 0s - loss: 2.0739e-05
Epoch 26: val_loss did not improve from 0.00031
314/314 [==============================] - 7s 21ms/step - loss: 2.0778e-05 - val_loss: 7.8810e-04
Epoch 2

Epoch 4/100
340/341 [============================>.] - ETA: 0s - loss: 2.5165e-05
Epoch 4: val_loss improved from 0.00090 to 0.00066, saving model to model\tmp_checkpoint.h5
341/341 [==============================] - 10s 30ms/step - loss: 2.5124e-05 - val_loss: 6.6239e-04
Epoch 5/100
341/341 [==============================] - ETA: 0s - loss: 2.0213e-05
Epoch 5: val_loss did not improve from 0.00066
341/341 [==============================] - 10s 29ms/step - loss: 2.0213e-05 - val_loss: 9.2278e-04
Epoch 6/100
340/341 [============================>.] - ETA: 0s - loss: 2.8388e-05
Epoch 6: val_loss improved from 0.00066 to 0.00065, saving model to model\tmp_checkpoint.h5
341/341 [==============================] - 11s 32ms/step - loss: 2.8369e-05 - val_loss: 6.4799e-04
Epoch 7/100
341/341 [==============================] - ETA: 0s - loss: 1.6430e-05
Epoch 7: val_loss improved from 0.00065 to 0.00057, saving model to model\tmp_checkpoint.h5
341/341 [==============================] - 11s 33ms/

341/341 [==============================] - 12s 34ms/step - loss: 8.5018e-06 - val_loss: 1.1213e-04
Epoch 38/100
341/341 [==============================] - ETA: 0s - loss: 1.0151e-05
Epoch 38: val_loss did not improve from 0.00009
341/341 [==============================] - 11s 33ms/step - loss: 1.0151e-05 - val_loss: 1.5163e-04
Epoch 39/100
341/341 [==============================] - ETA: 0s - loss: 9.2134e-06
Epoch 39: val_loss did not improve from 0.00009
341/341 [==============================] - 12s 35ms/step - loss: 9.2134e-06 - val_loss: 9.7561e-05
Epoch 40/100
341/341 [==============================] - ETA: 0s - loss: 8.5214e-06
Epoch 40: val_loss did not improve from 0.00009
341/341 [==============================] - 11s 33ms/step - loss: 8.5214e-06 - val_loss: 1.2045e-04
Epoch 41/100
340/341 [============================>.] - ETA: 0s - loss: 8.0385e-06
Epoch 41: val_loss did not improve from 0.00009
341/341 [==============================] - 11s 32ms/step - loss: 8.0318e-06 - va

Epoch 72/100
341/341 [==============================] - ETA: 0s - loss: 6.2461e-06
Epoch 72: val_loss did not improve from 0.00006
341/341 [==============================] - 10s 30ms/step - loss: 6.2461e-06 - val_loss: 6.3920e-05
Epoch 73/100
340/341 [============================>.] - ETA: 0s - loss: 6.5502e-06
Epoch 73: val_loss did not improve from 0.00006
341/341 [==============================] - 9s 28ms/step - loss: 6.5459e-06 - val_loss: 9.1717e-05
Epoch 74/100
340/341 [============================>.] - ETA: 0s - loss: 6.9286e-06
Epoch 74: val_loss did not improve from 0.00006
341/341 [==============================] - 10s 28ms/step - loss: 6.9210e-06 - val_loss: 6.9571e-05
Epoch 75/100
340/341 [============================>.] - ETA: 0s - loss: 6.1183e-06
Epoch 75: val_loss did not improve from 0.00006
341/341 [==============================] - 9s 27ms/step - loss: 6.1169e-06 - val_loss: 6.8676e-05
Epoch 76/100
340/341 [============================>.] - ETA: 0s - loss: 6.3556e-06

In [4]:
print(predict)

[array([[183115.75]], dtype=float32), array([[191497.62]], dtype=float32), array([[188854.48]], dtype=float32), array([[198333.03]], dtype=float32), array([[197163.42]], dtype=float32), array([[191829.33]], dtype=float32), array([[198086.92]], dtype=float32), array([[199661.53]], dtype=float32), array([[189898.66]], dtype=float32), array([[179386.31]], dtype=float32)]
